# Example 9: $\chi \chi \to b \bar{b}$

In this notebook, we'll show you how to use the code to obtain the temperature and ionization histories for a dark matter particle with mass $m_\text{DM}$ that annihilates/decays into a pair of Standard Model particles with some (redshift-dependent) rate, characterized by either a thermally-averaged $s$-wave annihilation cross section $\langle \sigma v \rangle$ or a decay lifetime $\tau$. 

## Initialization

First, let's initialize some of the modules that are required. In order to perform the calculation, we need to point the code to a set of transfer functions that determine how high-energy particles in our code cool. 

The user must first download the data files located [*here*](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DUOUWA). When these files are required, the user will be prompted for the directory in which they are stored. To avoid this prompt, the directory can be entered into `data_path` defined in the [*config*](https://darkhistory.readthedocs.io/en/master/_autosummary/config.html) module. 

In [ ]:
%load_ext autoreload
import sys
sys.path.append("..")

In [ ]:
%matplotlib inline

In [ ]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np

import darkhistory.physics as phys

import darkhistory.main as main

## Structure Formation

Dark matter annihilation rates are significantly enhanced after the formation of structures, owing to the fact that $\langle \rho^2 \rangle$ becomes greater than $\langle \rho_\text{DM} \rangle^2$, where $\rho_\text{DM}$ is the mass density of dark matter. To account for this increase in annihilation rate, boost factors, defined as

$$ 1 + \mathcal{B}(z) \equiv \frac{\langle \rho_\text{DM}^2 \rangle}{\langle \rho_\text{DM} \rangle^2} $$

have been computed under a wide variety of assumptions in the literature. The boost factor allows us to write the energy injection rate as

$$ \left( \frac{dE}{dV \, dt} \right)^\text{inj} = [1 + \mathcal{B}(z)] \left( \frac{dE}{dV \, dt} \right)^\text{inj}_\text{smooth} \,,$$

where the subscript 'smooth' indicates the smooth energy injection rate with the average mass density of dark matter assumed. 

In DarkHistory, we provide the 4 $s$-wave boost factors calculated in [[1]](#cite_2016), and shown in Fig. 2. We also include the $\text{erfc}$ model provided in [[2]](#cite_erfc), Eq. (2.7). The user can obtain these boost factors as *functions* that themselves return $1 + \mathcal{B}(z)$ as a function of redshift $1+z$, using [*struct_boost_func()*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/physics/darkhistory.physics.struct_boost_func.html) in [*physics*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/physics). See the documentation for details on how to use this function. Below, we use the "Einasto with substructure" model in [[1]](#cite_2016).

In [ ]:
struct_boost = phys.struct_boost_func(model='einasto_subs')
struct_boost_NFW = phys.struct_boost_func(model='NFW_no_subs')

Let's plot this structure formation prescription to make sure that we have the right behavior. We'll make a plot of the effective density

$$ \rho_\text{eff} \equiv \sqrt{1 + \mathcal{B}(z)} \langle \rho_\text{DM} \rangle =  \sqrt{1 + \mathcal{B}(z)} \rho_{\text{DM},0} (1+z)^3 $$

where $\rho_{\text{DM},0}$ is the dark matter density today, which is stored in [*physics.rho_DM()*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/physics/darkhistory.physics.rho_DM.html). 

This plot should be directly compared with [[1]](#cite_2016). 

In [ ]:
plt.figure()

plt.loglog()

z_vec = 10**np.arange(-1, 2, 0.005)
# Note that struct_boost accepts 1+z as the argument, which is the default
# way in which we pass redshift to functions in DarkHistory. 
rho_eff_vec = np.sqrt(struct_boost(1+z_vec)) * phys.rho_DM * (1+z_vec)**3 
rho_eff_vec_NFW = np.sqrt(struct_boost_NFW(1+z_vec)) * phys.rho_DM * (1+z_vec)**3 

# Convert to GeV/cm^3
rho_eff_vec /= 1e9
rho_eff_vec_NFW /= 1e9

rho_eff_plot,    = plt.plot(z_vec, rho_eff_vec, label=r'$\rho_\mathrm{eff}$, Einasto with Sub.')
rho_eff_plot_NFW,    = plt.plot(z_vec, rho_eff_vec_NFW, label=r'$\rho_\mathrm{eff}$, NFW without Sub.')
smooth_rho_plot, = plt.plot(z_vec, phys.rho_DM * (1+z_vec)**3 / 1e9, 'k--', label=r'Smooth Dark Matter')

plt.axis([0.1, 70, 1e-6, 2])
plt.xlabel('Redshift $z$')
plt.ylabel(r'Density [GeV cm$^{-3}$]')
plt.title('Structure Formation Boost')

plt.legend(handles=[smooth_rho_plot, rho_eff_plot, rho_eff_plot_NFW])

##  $b\bar{b}$ annihilation spectra

In the model we consider below, dark matter annihilates into b-quark/anti-quark pairs, $\chi\chi \to b \bar{b}$, which then resolve into spectra of stable particles.  As descibed in Section II of the paper, we need only consider the photon and electron/positron spectra, because the other stable particles (neutrinos, proton/anti-protons, heavier nuclei) are either ineffective at depositing their energy into the IGM or are produced in negligible amounts.

Below we plot the resulting photon and electron/positron spectra coming from $\chi\chi \to b \bar{b}$.

In [ ]:
from darkhistory.spec.pppc import get_pppc_spec

In [ ]:
eng = np.exp(np.array(
    [-9.171782342409985 + 0.07711606*i for i in np.arange(500)]
))
phot_spec = get_pppc_spec(50e9, eng, 'b', 'phot', decay=False)
elec_spec = get_pppc_spec(50e9, eng, 'b', 'elec', decay=False)

In [ ]:
fig_bbbar = plt.figure(figsize=(15.5,6.5))

ax = plt.subplot(1,2,1)

ax.loglog()

plt.plot(phot_spec.eng, phot_spec.eng**2 * phot_spec.dNdE)

plt.axis([10, 80e9, 1e-29, 1e11])

plt.title(r'Injected Photon Spectrum')
plt.xlabel(r'Kinetic Energy $E$ [eV]')
plt.ylabel(r'$E^2 \; \frac{d\mathbf{N}^\gamma_\mathrm{inj}}{dE}$')

plt.text(0.06, 0.88, r'$m_\chi$ = 50 GeV', fontsize=20, transform=ax.transAxes)
plt.text(0.09, 0.82, r'$\chi\chi \to b \bar{b}$', fontsize=20, transform=ax.transAxes)



#################

ax = plt.subplot(1,2,2)

ax.loglog()

plt.plot(elec_spec.eng, elec_spec.eng**2 * elec_spec.dNdE)

plt.axis([10, 80e9, 1e-8, 1e11])

plt.text(0.06, 0.88, r'$m_\chi$ = 50 GeV', fontsize=20, transform=ax.transAxes)
plt.text(0.09, 0.82, r'$\chi\chi \to b \bar{b}$', fontsize=20, transform=ax.transAxes)


plt.title(r'Injected $e^+e^-$ Spectrum')
plt.xlabel(r'Kinetic Energy $E$ [eV]')
plt.ylabel(r'$E^2 \; \frac{d\mathbf{N}^e_\mathrm{inj}}{dE}$')

## $\chi \chi \to b \bar{b}$ Annihilation

We are now ready to obtain the ionization and thermal history for a dark matter model with $m_\chi = $ 50 GeV, and a thermally-averaged annihilation cross section of $\langle \sigma v \rangle = 2 \times 10^{-26} \mathrm{ cm}^{3} s^{-1}$. This model has been found to be consistent with the galactic center excess [[3]](#cite_excess) and the AMS-02 cosmic ray antiproton spectrum [[4]](#cite_AMS). 

The function we will use is [*main.evolve()*](https://darkhistory.readthedocs.io/en/master/_autosummary/main/main.evolve.html). The function is called with the following arguments:

```python
def evolve(
    in_spec_elec=None, in_spec_phot=None,
    rate_func_N=None, rate_func_eng=None,
    DM_process=None, mDM=None, sigmav=None, lifetime=None, primary=None,
    struct_boost=None,
    start_rs=None, end_rs=4, helium_TLA=False,
    reion_switch=False, reion_rs=None,
    photoion_rate_func=None, photoheat_rate_func=None, xe_reion_func=None,
    init_cond=None, coarsen_factor=1, backreaction=True, 
    compute_fs_method='no_He', mxstep=1000, rtol=1e-4,
    use_tqdm=True
)
```

A full accounting of how to use all of the functions can be found throughout the examples given here and the documentation provided. Here we provide just a brief summary:

* `in_spec_elec`, `in_spec_phot`, `rate_func_N` and `rate_func_eng` are used to specify input electron and photon spectra per energy injection event, occurring at some event and energy injection rate. This allows the user to define energy injection processes of any type; 

* `DM_process`, `mDM`, `sigmav`, `lifetime`, `primary` and `struct_boost` are used for dark matter models with annihilation/decay with some mass, $\langle \sigma v \rangle$ or lifetime, forming some Standard Model primary particle and with some structure formation prescription;

* `start_rs` and `end_rs` to define the redshift boundaries of the problem (`start_rs` needs to be specified for DM processes, but not if `in_spec_elec` or `in_spec_phot` is in use, which should have its redshift initialized instead);

* `helium_TLA` determines whether to include helium or neglect it;

* `reion_switch`, `reion_rs`, `photoion_rate_func`, `photoheat_rate_func`, `xe_reion_func` are all passed to `darkhistory.tla.get_history` for purposes of modeling reionization; 

* `init_cond` allows the user to specify initial conditions in temperature and ionization levels that are not the baseline conditions at the starting redshift; 

* `coarsen_factor` determines how much coarsening to use; 

* `backreaction` controls whether $f_c(z)$ are evaluated using the baseline ionization levels, or if they are evaluated based on the calculated ionization levels, which would include the backreaction effect; 

* `compute_fs_method` controls how $f_c(z)$ is calculated; 

* `mxstep` and `rtol` are parameters that can be used to control the ODE solver, and

* `use_tqdm` controls whether to use the `tqdm` progress bar or not. 

When called for the first time, the data files that are downloaded as part of DarkHistory are loaded into memory for use by [*main.evolve()*](https://darkhistory.readthedocs.io/en/master/_autosummary/main/main.evolve.html). The successful import of these files will generate the following output:

```
****** Loading transfer functions... ******
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******
```

Once the files have been properly loaded, the calculation will begin.

Here is how we call the function to evaluate the ionization and temperature history for our dark matter model, *without* backreaction (we will run it with `coarsen_factor = 32` for speed). We do not include any reionization in this calculation, and also neglect helium here. 

In [ ]:
bbbar_noBR = main.evolve(
    DM_process='swave', mDM=50e9, sigmav=2e-26, primary='b',
    start_rs = 3000.,
    coarsen_factor=32, backreaction=False,
    struct_boost=struct_boost
)

The output is a dictionary with the following keys:

* `'rs'` -- the redshift abscissa of the outputs. 
* `'x'`  -- ($x_\text{HII}$, $x_\text{HeII}$) at each redshift point. 
* `'Tm'` -- $T_m$ at each redshift point. 
* `'highengphot'`, `'lowengphot'` and `'lowengelec'` -- the high-energy photon, low-energy photon and low-energy electron spectra at each step. 
* `'cmbloss'` -- the energy of CMB photons upscattered per unit time in each step. 
* `'f'`  -- A dictionary containing all the $f_c(z)$ values from both the low-energy spectra (`'low'`) and the high-energy electron deposition estimate (`'high'`). 

Next, let's run the same result *with* backreaction included.

In [ ]:
bbbar_BR = main.evolve(
    DM_process='swave', mDM=50e9, sigmav=2e-26, primary='b',
    start_rs = 3000,
    coarsen_factor=32, backreaction=True,
    struct_boost=struct_boost
)

Now let's make a plot comparing these two results. Pull out the relevant information first:

In [ ]:
# Redshift abscissa. Identical for both cases. 
rs_vec = bbbar_noBR['rs']

# Tm, stored in eV and converted to K. 
Tm_noBR = bbbar_noBR['Tm']/phys.kB
Tm_BR   = bbbar_BR['Tm']/phys.kB

# xHII. 
xHII_noBR = bbbar_noBR['x'][:,0]
xHII_BR   = bbbar_BR['x'][:,0]

And now the plotting!

In [ ]:
fig_bbbar = plt.figure(figsize=(15,6.5))

ax = plt.subplot(1,2,1)

ax.loglog()

plot_Tm_noBR,   = plt.plot(rs_vec, Tm_noBR, label='No Backreaction')
plot_Tm_BR, = plt.plot(rs_vec, Tm_BR, label='With Backreaction')
# Plot the baseline result for comparison
plot_Tm_std,    = plt.plot(rs_vec, phys.Tm_std(rs_vec)/phys.kB, 'k--', label='Baseline')

plt.title(r'\bf{Temperature History}')
plt.xlabel(r'Redshift $(1+z)$')
plt.ylabel(r'Matter Temperature $T_m$ [K]')

plt.axis([4, 1e2, 0.3, 3e2])

# Text
plt.text(0.06, 0.88, r'$\chi \chi \to b \bar{b}$, $m_\chi$ = 50 GeV', fontsize=20, transform=ax.transAxes) 
plt.text(0.06, 0.82, r'$\langle \sigma v \rangle = 2 \times 10^{-26}$ cm$^3$ s$^{-1}$', fontsize=20, transform=ax.transAxes) 

leg = plt.legend(handles=[plot_Tm_noBR, plot_Tm_BR, plot_Tm_std], loc=4)

#################

ax = plt.subplot(1,2,2)

ax.loglog()

plot_xHII_noBR,   = plt.plot(rs_vec, xHII_noBR, label='No Backreaction')
plot_xHII_BR, = plt.plot(rs_vec, xHII_BR, label='With Backreaction')
# Plot the baseline result for comparison
plot_xHII_std,    = plt.plot(rs_vec, phys.xHII_std(rs_vec), 'k--', label='Baseline')

plt.title(r'\bf{Ionization History}')
plt.xlabel(r'Redshift $(1+z)$')
plt.ylabel(r'Hydrogen Ionization Fraction $x_\mathrm{HII}$')

plt.axis([4, 1e2, 0.8e-4, 2e-3])

# Text
plt.text(0.06, 0.88, r'$\chi \chi \to b \bar{b}$, $m_\chi$ = 50 GeV', fontsize=20, transform=ax.transAxes) 
plt.text(0.06, 0.82, r'$\langle \sigma v \rangle = 2 \times 10^{-26}$ cm$^3$ s$^{-1}$', fontsize=20, transform=ax.transAxes) 

leg = plt.legend(handles=[plot_xHII_noBR, plot_xHII_BR, plot_xHII_std], loc=4)

Although the temperature difference with and without backreaction is only about 50\% at $1+z = 4$, this is of course simply one example chosen to highlight how easy it is to get the temperature/ionization histories for any simple dark matter model. Models with decaying dark matter, for example, can lead to much larger differences. We will see an instance of this in a later example.

## Details

Importing `main` also imports a list of interpolation functions over various transfer functions. The first group of transfer functions involve propagation and deposition transfer functions that multiply a spectrum of propagating photons from a given redshift to give various quantities at the next redshift step. These include propagating photons (`highengphot_interp_tf`), low-energy photons (`lowengphot_interp_tf`), low-energy electrons (`lowengelec_interp_tf`), the energy of CMB photons upscattered per unit time (`CMB_engloss_interp`) and high-energy deposition (`highengdep_interp`). These interpolation functions are of type `TransferFuncInterps`, under the module `darkhistory.spec.transferfunclist`. See the documentation for more details on how to use these objects.

Together with these, there are also transfer function matrices that multiply a spectrum of electrons to give the spectrum of scattered photons per unit time in the Thomson (`ics_thomson_ref_tf`) and relativistic (`ics_rel_ref_tf`) regimes, as well as the energy loss spectrum per unit time (`ics_engloss_ref_tf`), i.e. number of photons which lose some amount of energy per unit time. These are used to work out how injected electrons cool in `main.evolve()`. 

## Bibliography

[1]<a id='cite_2016'></a> Hongwan Liu, Tracy R. Slatyer, and Jesús Zavala, “Contributions to cosmic reionization from dark matter annihilation and decay,” Phys. Rev. D94, 063507 (2016), arXiv:1604.02457 [astro-ph.CO].

[2]<a id='cite_erfc'></a> Carmelo Evoli, Andrei Mesinger, and Andrea Ferrara, “Unveiling the nature of dark matter with high redshift 21 cm line experiments,” JCAP 1411, 024 (2014), arXiv:1408.1109 [astro-ph.HE].

[3]<a id='cite_excess'></a> Francesca Calore, Ilias Cholis, Christopher McCabe, and Christoph Weniger, “A Tale of Tails: Dark Matter Inter- pretations of the Fermi GeV Excess in Light of Back- ground Model Systematics,” Phys. Rev. D91, 063003 (2015), arXiv:1411.4647 [hep-ph]

[4]<a id='cite_AMS'></a> Ming-Yang Cui, Qiang Yuan, Yue-Lin Sming Tsai, and Yi-Zhong Fan, “Possible dark matter annihilation signal in the AMS-02 antiproton data,” Phys. Rev. Lett. 118, 191101 (2017), arXiv:1610.03840 [astro-ph.HE].